# CREACION DEL DATAMART ANALITICO

## SET UP 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


#Automcompletar rápido
%config IPCompleter.greedy=True

#Formato de display
pd.options.display.float_format = '{:15.2f}'.format

## CARGA DE DATOS 

In [2]:
import sqlalchemy as sa
con = sa.create_engine('sqlite:///../Datos/ecommerce.db')

In [3]:
from sqlalchemy import inspect
insp = inspect(con)
tablas = insp.get_table_names()
tablas

['2019-Dec', '2019-Nov', '2019-Oct', '2020-Feb', '2020-Jan']

In [4]:
oct = pd.read_sql('2019-Oct', con)
nov = pd.read_sql('2019-Nov', con)
dic = pd.read_sql('2019-Dec', con)
ene = pd.read_sql('2020-Jan', con)
feb = pd.read_sql('2020-Feb', con)

## INTEGRACIÓN DE DATOS 

In [5]:
df = pd.concat([oct,nov,dic,ene,feb], axis = 0)
df

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,68,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,72,2019-10-01 00:01:55 UTC,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,95,2019-10-01 00:02:50 UTC,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,122,2019-10-01 00:03:41 UTC,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,124,2019-10-01 00:03:44 UTC,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...,...
429785,4156660,2020-02-29 23:58:49 UTC,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,4156663,2020-02-29 23:58:57 UTC,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,4156668,2020-02-29 23:59:05 UTC,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,4156675,2020-02-29 23:59:28 UTC,view,5817692,1487580010872045658,None,None,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


## CALIDAD DE DATOS 

### Tipos de variables

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095076 entries, 0 to 429789
Data columns (total 10 columns):
 #   Column         Dtype  
---  ------         -----  
 0   index          int64  
 1   event_time     object 
 2   event_type     object 
 3   product_id     int64  
 4   category_id    int64  
 5   category_code  object 
 6   brand          object 
 7   price          float64
 8   user_id        int64  
 9   user_session   object 
dtypes: float64(1), int64(4), object(5)
memory usage: 175.8+ MB


Se eliminará la columna index.

In [7]:
df.drop(columns = 'index', inplace = True)

Análisis y corrección de tipos.

* Convertir `event_time` a datetime

In [9]:
#forma avanzada crando una función
# (tarda bastante menos tiempo en ejecutar que convirtiendo de forma tradicional)

def datetime_rapido(dt,formato):

    def divide_fecha(fecha):
        division = fecha.split()
        date = division[0]
        time = division[1]
        cadena = date + ' ' + time
        return cadena

    resultado = pd.to_datetime(dt.apply(lambda x: divide_fecha(x)), format = formato)

    return resultado

Ejecutamos la función.

In [10]:
formato = '%Y-%m-%d %H:%M:%S'

df.event_time = datetime_rapido(df.event_time,formato)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2095076 entries, 0 to 429789
Data columns (total 9 columns):
 #   Column         Dtype         
---  ------         -----         
 0   event_time     datetime64[ns]
 1   event_type     object        
 2   product_id     int64         
 3   category_id    int64         
 4   category_code  object        
 5   brand          object        
 6   price          float64       
 7   user_id        int64         
 8   user_session   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 159.8+ MB


### Nombres de variables

Renombramos las variables a español.

In [12]:
df.columns = ['fecha',
              'evento',
              'producto',
              'categoria',
              'categoria_cod',
              'marca',
              'precio',
              'usuario',
              'sesion']
df

,fecha,evento,producto,categoria,categoria_cod,marca,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,None,None,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


### Análisis de nulos

In [13]:
df.isna().sum().sort_values(ascending = False)

categoria_cod    2060411
marca             891646
sesion               506
fecha                  0
evento                 0
producto               0
categoria              0
precio                 0
usuario                0
dtype: int64

Conclusiones:

* `categoria_cod` tiene casi todos los registros nulos.
* `marca` tiene casi la mitad de los registros nulos.
* Hay 500 nulos en `sesión`.

Acciones:

* Eliminar las variables `categoria_cod` y `marca`.
* Eliminar los nulos de sesión ya que es una variable relevante.

In [14]:
df = df.drop(columns = ['categoria_cod','marca']).dropna()
df

,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


### Análisis de las variables numéricas

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
producto,2094570.00,5487103.56,1300923.90,3752.00,5724652.00,5811665.00,5858353.00,5932595.00
categoria,2094570.00,1553112489392098048.00,167907497920480576.00,1487580004807082752.00,1487580005754995456.00,1487580008246412288.00,1487580013489291520.00,2242903426784559104.00
precio,2094570.00,8.42,19.14,-47.62,2.05,4.00,6.86,327.78
usuario,2094570.00,521077545.56,87553855.76,4661182.00,480613387.00,553341613.00,578406571.00,622087993.00


Se observan valores negativos en el precio. Se analizará con más profundidad.

In [16]:
df[df.precio <= 0]

,fecha,evento,producto,categoria,precio,usuario,sesion
343,2019-10-01 02:15:41,view,5892052,1487580010377117763,0.00,555455025,320f6021-30ac-4a58-ae17-bac1cc32aac3
924,2019-10-01 05:16:30,view,5889621,1487580010561667147,0.00,523988665,00849bd2-fcd2-4cb4-af31-4e264f151848
933,2019-10-01 05:18:03,view,5889622,1487580010561667147,0.00,523988665,80cfe614-f0a5-4101-a2b6-a21227590470
937,2019-10-01 05:18:46,view,5889623,1487580010561667147,0.00,523988665,c2cd0464-3d2b-48e2-9667-bac248fe297a
1077,2019-10-01 05:38:01,view,5889627,1487580010561667147,0.00,523988665,8b2bf9d8-43f0-43b2-bed3-13b2c956cada
...,...,...,...,...,...,...,...
428011,2020-02-29 20:04:49,cart,5824841,1897124478404526487,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46
428012,2020-02-29 20:04:49,cart,5826413,1487580005511725929,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46
428013,2020-02-29 20:04:49,cart,5832437,1487580007675986893,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46
428014,2020-02-29 20:04:49,cart,5851606,2055161088059638328,0.00,469761446,8bf369b4-92c0-4fb8-88a5-8a2dd0947e46


Son unos 20000 registros, que podrían eliminarse.

Pero antes ¿se concentran quizá en algún producto determinado?

In [17]:
df[df.precio <= 0].producto.value_counts().head(10)

5896186    79
5903915    50
5873428    37
5851294    29
5851304    29
5837624    28
5712583    27
5851272    27
5899512    26
5907812    26
Name: producto, dtype: int64

No parece que sea problema de un producto concreto, así que se eliminarán dichos registros.

In [18]:
df = df[df.precio > 0]
df

,fecha,evento,producto,categoria,precio,usuario,sesion
0,2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
429785,2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,2020-02-29 23:59:28,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


### Análisis de las variables categóricas

In [19]:
df.evento.nunique()

4

In [20]:
df.evento.value_counts()

view                961558
cart                574547
remove_from_cart    410357
purchase            127564
Name: evento, dtype: int64

In [21]:
df.producto.nunique()

45327

In [22]:
df.categoria.nunique()

508

### Índice

Se establece la fecha como índice del dataframe.

In [23]:
df.set_index('fecha', inplace = True)
df

,evento,producto,categoria,precio,usuario,sesion
fecha,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90


## TRANSFORMACION DE DATOS

Se crearán 3 nuevos tipos de variables:
* Componentes de la fecha.
* Variables de calendario: Festivos locales (Rusia).
* Indicadores exógenos: Días no necesariamente festivos pero con interés comercial, como Black Friday, Cyber Monday, Reyes, San Valentín.

### Componentes de la fecha

In [24]:
def componentes_fecha(dataframe):
    date = dataframe.index.date
    año = dataframe.index.year
    mes = dataframe.index.month
    dia = dataframe.index.day
    hora = dataframe.index.hour
    minuto = dataframe.index.minute
    segundo = dataframe.index.second
    
    
    return(pd.DataFrame({'date':date, 'año':año,'mes':mes, 'dia':dia, 'hora':hora, 'minuto':minuto, 'segundo':segundo}))

In [25]:
df = pd.concat([df.reset_index(),componentes_fecha(df)], axis = 1).set_index('fecha')
df

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo
fecha,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,49
2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,57
2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,59,5


### Variables de calendario: festivos

Para incorporar festivos se empleará el paquete holidays.

No es perfecto, pero da mucha flexibilidad dado que contiene festivos de varios países e incluso a nivel comunidades.

Instalación con: 
```
conda install -c conda-forge holidays
```

Listado de países y el uso básico en:

https://github.com/dr-prodigy/python-holidays

Definición del objeto festivo_ru ya que el cliente es un ecommerce Ruso.

In [27]:
festivo_ru = holidays.RU(years=2020)
festivo_ru

{datetime.date(2020, 1, 1): 'Новый год',
 datetime.date(2020, 1, 2): 'Новый год',
 datetime.date(2020, 1, 3): 'Новый год',
 datetime.date(2020, 1, 4): 'Новый год',
 datetime.date(2020, 1, 5): 'Новый год',
 datetime.date(2020, 1, 6): 'Новый год',
 datetime.date(2020, 1, 7): 'Православное Рождество',
 datetime.date(2020, 1, 8): 'Новый год',
 datetime.date(2020, 2, 23): 'День защитника отечества',
 datetime.date(2020, 3, 8): 'День женщин',
 datetime.date(2020, 5, 1): 'Праздник Весны и Труда',
 datetime.date(2020, 5, 9): 'День Победы',
 datetime.date(2020, 6, 12): 'День России',
 datetime.date(2020, 11, 4): 'День народного единства'}

Flag para indicar en cada registro si era un día festivo o no.

In [31]:
df['festivo'] = df.date.apply(lambda x: 1 if (x in festivo_ru) else 0)
df

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo
fecha,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,0
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,0
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50,0
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41,0
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,49,0
2020-02-29 23:58:57,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,58,57,0
2020-02-29 23:59:05,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,2020-02-29,2020,2,29,23,59,5,0


Comprobación de los festivos.

In [32]:
df[df.festivo == 1].date.value_counts().sort_index()

2019-11-04    16430
2020-01-01     7644
2020-01-02    10776
2020-01-03    10617
2020-01-04    13084
2020-01-05    14554
2020-01-06    10621
2020-01-07    12922
2020-01-08    14004
2020-02-23     9817
Name: date, dtype: int64

### Indicadores exógenos

Creación de indicadores exógenos para Black Friday y San Valentín.

In [33]:
df['black_friday'] = 0
df.loc['2019-11-29','black_friday'] = 1

df['san_valentin'] = 0
df.loc['2020-02-14','san_valentin'] = 1

Comprobación.

In [34]:
df['black_friday'].value_counts()

0    2051695
1      22331
Name: black_friday, dtype: int64

In [35]:
df['san_valentin'].value_counts()

0    2061781
1      12245
Name: san_valentin, dtype: int64

## TABLÓN ANALITICO FINAL

In [36]:
df.head()

,evento,producto,categoria,precio,usuario,sesion,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,0,0,0
2019-10-01 00:01:55,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,0,0,0
2019-10-01 00:02:50,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,2019-10-01,2019,10,1,0,2,50,0,0,0
2019-10-01 00:03:41,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df,2019-10-01,2019,10,1,0,3,41,0,0,0
2019-10-01 00:03:44,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,2019-10-01,2019,10,1,0,3,44,0,0,0


Reorganizción de las columnas en un orden más natural.

In [37]:
variables = df.columns.to_list()
variables

['evento',
 'producto',
 'categoria',
 'precio',
 'usuario',
 'sesion',
 'date',
 'año',
 'mes',
 'dia',
 'hora',
 'minuto',
 'segundo',
 'festivo',
 'black_friday',
 'san_valentin']

In [38]:
orden = ['usuario',
         'sesion',
         'categoria',
         'evento',
         'producto',
         'precio']

orden

['usuario', 'sesion', 'categoria', 'evento', 'producto', 'precio']

In [39]:
resto = [nombre for nombre in variables if nombre not in orden]

resto

['date',
 'año',
 'mes',
 'dia',
 'hora',
 'minuto',
 'segundo',
 'festivo',
 'black_friday',
 'san_valentin']

In [40]:
df = df[orden + resto]
df

,usuario,sesion,categoria,evento,producto,precio,date,año,mes,dia,hora,minuto,segundo,festivo,black_friday,san_valentin
fecha,,,,,,,,,,,,,,,,
2019-10-01 00:01:46,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,1487580005092295511,view,5843665,9.44,2019-10-01,2019,10,1,0,1,46,0,0,0
2019-10-01 00:01:55,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,1487580013069861041,cart,5868461,3.57,2019-10-01,2019,10,1,0,1,55,0,0,0
2019-10-01 00:02:50,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770,1487580006300255120,view,5877456,122.22,2019-10-01,2019,10,1,0,2,50,0,0,0
2019-10-01 00:03:41,555448072,b5f72ceb-0730-44de-a932-d16db62390df,1487580013749338323,view,5649270,6.19,2019-10-01,2019,10,1,0,3,41,0,0,0
2019-10-01 00:03:44,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5,1487580005411062629,view,18082,16.03,2019-10-01,2019,10,1,0,3,44,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-29 23:58:49,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,cart,5815662,0.92,2020-02-29,2020,2,29,23,58,49,0,0,0
2020-02-29 23:58:57,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,view,5815665,0.59,2020-02-29,2020,2,29,23,58,57,0,0,0
2020-02-29 23:59:05,147995998,5ff96629-3627-493e-a25b-5a871ec78c90,1487580006317032337,cart,5815665,0.59,2020-02-29,2020,2,29,23,59,5,0,0,0


Guardado del tablón analítico en formato pickle para no perder los metadatos.

In [41]:
df.to_pickle('../Datos/tablon_analitico.pickle')